In [ ]:
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt

import shapely as sp
import shapely.geometry

In [ ]:
import polymetric as pm
reload(pm)
import polymetric.exporters.dxf
import polymetric.exporters.gds
import polymetric.plotting

In [ ]:
def generate_jbeam(**kw):
    # parameters
    hole_b = kw.pop("hole_b", 0.750/2.0)
    hole_a = kw.pop("hole_a", 0.270/2.0)

    slit_d = kw.pop("slit_d", 0.045)
    beam_w = kw.pop("beam_w", 0.900)
    beam_l = kw.pop("beam_l", 10.0)

    holes_n = kw.pop("holes_n", 11)

    defect_d = kw.pop("defect_d", 0.400)
    lattice_d = kw.pop("lattice_d", 0.450)
    
    # actual construction of geometry
    hole = pm.Ellipse(a=hole_a, b=hole_b, x=defect_d/2)
    hole_sweep = pm.ParametricSweep(
        constructor=hole.clone,
        sweep_over=np.arange(holes_n),
        sweep_params = {
            'x': lambda n: defect_d/2 + n*lattice_d,
#             'b': lambda n: hole_b if n > 2 else (0.7 + 0.1*n)*hole_b,
#             'a': lambda n: hole_a if n > 2 else (0.7 + 0.1*n)*hole_a,
        }
    )
    hole_sweep_mirror = pm.Scaled(hole_sweep, scales=(-1.0, 1.0))
    all_holes = pm.Combined([hole_sweep, hole_sweep_mirror])
    full_beam = pm.Rectangle(x=0.0, y=0.0, w=beam_l, h=beam_w)
    minus_holes = pm.Difference([full_beam, all_holes])
    minus_slit = pm.Difference([minus_holes, pm.Rectangle(x=0.0, y=0.0, w=beam_l, h=slit_d)])
    
    return minus_slit

In [ ]:
beam_polys = []

for i in range(1,12):
    poly = generate_jbeam(holes_n=i)
    pm.plotting.inspect_shapes(poly)
    pm.exporters.dxf.save(poly, "test_generated_files/beam_n_%d.dxf" % i)
    pm.exporters.gds.save(poly, "test_generated_files/beam_n_%d.gds" % i)
    beam_polys.append(poly)

In [ ]:
poly.get_exterior_vertex_lists()